In [1]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from PIL import Image

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

### 加载模型

加载生成器模型

In [2]:
class Generator(nn.Module):
    def __init__(self, input_dim=100, output_dim=1, class_num=10):
        '''
        初始化生成网络
        :param input_dim:输入随机噪声的维度，（随机噪声是为了增加输出多样性）
        :param output_dim:生成图像的通道数（灰度图为1，RGB图为3）
        :param class_num:图像种类
        '''
        super(Generator, self).__init__()
        """
         为什么需要拼接随机噪声和条件向量？
         拼接随机噪声和条件向量的目的是将两种信息结合起来，作为生成器的输入：
         随机噪声：提供生成数据的随机性。
         条件向量：提供生成数据的条件信息。
         通过拼接，生成器可以根据条件向量生成符合特定条件的数据, 同时确保每次生成的数据会有所不同
         """
        self.input_dim = input_dim
        self.class_num = class_num
        self.output_dim = output_dim
        
        # 嵌入层处理条件向量(类别标签), 提高条件信息的表达能力
        self.label_emb = nn.Embedding(class_num, class_num)
        
        # 全连接层，将输入向量映射到高维空间，然后通过反卷积层生成图像
        self.fc = nn.Sequential(
            nn.Linear(self.input_dim + self.class_num, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, 128 * 7 * 7),
            nn.BatchNorm1d(128 * 7 * 7),
            nn.LeakyReLU(0.2, inplace=True)
        )

        # 反卷积层（转置卷积层），用于将高维特征图逐步上采样为最终图像
        self.deconv = nn.Sequential(
            nn.ConvTranspose2d(128, 128, 4, 2, 1),  # 7x7 -> 14x14
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1),   # 14x14 -> 28x28
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, self.output_dim, 3, 1, 1),  # 保持尺寸不变，但细化特征
            nn.Tanh()  # 激活函数，将输出值限制在 [-1, 1] 范围内，适合生成图像
        )
 
    def forward(self, noise, labels):
        # 标签处理
        label_embedding = self.label_emb(labels)
        
        # 拼接噪声和条件向量
        x = torch.cat([noise, label_embedding], dim=1)
        
        # 通过全连接层
        x = self.fc(x)
        
        # 重塑为特征图
        x = x.view(-1, 128, 7, 7)
        
        # 通过反卷积层生成图像
        x = self.deconv(x)
        
        return x

generator = Generator().to(device)
model_path = '../models/4_GAN_Image_Generator/MINIST_generator.pth'
generator.load_state_dict(torch.load(model_path))

C:\Users\PC\AppData\Local\Temp\ipykernel_26836\4019217968.py:69: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  generator.load_state_dict(torch.load(model_path))


<All keys matched successfully>

加载数字识别模型，用于辅助数字图像生成

In [3]:
class Digit_recognizer(torch.nn.Module):
    def __init__(self):
        # （batch,1,28,28）
        super(Digit_recognizer, self).__init__()
        self.conv1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3), #（batch,32,26,26）
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2), #（batch,32,13,13）
        )
        self.conv2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3), #（batch,64,11,11）
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2), #（batch,64,5,5）
        )
        self.fc = torch.nn.Sequential(
            torch.nn.Linear(1600, 50),
            # torch.nn.ReLU(),  # 添加ReLU激活函数
            torch.nn.Linear(50, 10),
        )

    def forward(self, x):
        batch_size = x.size(0)
        x = self.conv1(x)  # 一层卷积层,一层池化层,一层激活层
        x = self.conv2(x)  # 再来一次
        x = x.view(batch_size, -1)  # flatten 变成全连接网络需要的输入
        x = self.fc(x)
        return x  # 最后输出的是维度为10的，也就是（对应数学符号的0~9）

digit_recognizer = Digit_recognizer().to(device)
model_path = '../models/1_Handwritten_Digit_Recognition/model_weights.pth'
digit_recognizer.load_state_dict(torch.load(model_path))

C:\Users\PC\AppData\Local\Temp\ipykernel_26836\3895384144.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  digit_recognizer.load_state_dict(torch.load(model_path))


<All keys matched successfully>

### 手写数字图像生成

In [4]:
def generate_digit_image(generator, digit):
    """
    生成指定数字的图片
    :param generator: 训练好的生成器模型
    :param digit: 要生成的数字 (0-9)
    :return: 生成的图片 (PIL 图像)
    """
    generator.eval()  # 设置为评估模式
    with torch.no_grad():
        # 生成随机噪声
        noise = torch.randn(1, generator.input_dim).to(device)
        
        # 创建标签
        label = torch.tensor([digit]).to(device)
        
        # 生成图片
        fake_image = generator(noise, label)
        
        # 将图片从 [-1, 1] 转换到 [0, 1]
        fake_image = (fake_image.squeeze().cpu() + 1) / 2.0
        
        # 将 2D 张量 (H, W) 转换为 3D 张量 (1, H, W)
        fake_image = fake_image.unsqueeze(0)
        
        # 转换为 PIL 图像
        fake_image = transforms.ToPILImage()(fake_image)
        
        return fake_image

# 预测数字
def predict_image(digit_recognizer, image):
    # 图像预处理
    transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=1),  # 转换为灰度
        transforms.Resize((28, 28)),                 # 调整到 28x28
        transforms.ToTensor(),                       # 转换为张量
        transforms.Normalize((0.5,), (0.5,))         # 归一化到 [-1, 1]
    ])
    image = transform(image)
    image = image.to(device)
    image = image.unsqueeze(0)
    
    with torch.no_grad():
        output = digit_recognizer(image)
        _, predicted = torch.max(output.data, 1)
    return predicted.item()

# 利用数字识别模型确保生成的数字图片更清晰
def generate_high_quality_digit_image(generator, digit_recognizer, digit):
    while True:
        digit_image = generate_digit_image(generator, digit)
        predict_label = predict_image(digit_recognizer, digit_image)
        if int(predict_label) == digit:
            return digit_image

generate_high_quality_digit_image(generator, digit_recognizer, 0)

In [6]:
# 生成一个包含 10x10 个不同数字的大图片，并保存到本地
plt.figure(figsize=(10, 10))  # 设置画布大小
plt.subplots_adjust(wspace=0.1, hspace=0.1)  # 调整子图间距

for i in range(10):  # 行
    for j in range(10):  # 列
        # 生成数字 j 的图片
        digit_image = generate_high_quality_digit_image(generator, digit_recognizer, j)
        
        # 将图片添加到子图中
        ax = plt.subplot(10, 10, i * 10 + j + 1)
        ax.imshow(digit_image, cmap='gray')
        ax.axis('off')  # 关闭坐标轴

save_path = './data/demo.png'
# 保存大图片
plt.savefig(save_path, bbox_inches='tight')
plt.close()
print(f"图片已保存到 {save_path}")

图片已保存到 ./data/demo.png
